In [1]:
# Imports
from biocrnpyler import *
from genelet import *
import pylab as plt
import numpy as np
from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import bokeh.io
import bokeh.plotting

from bioscrape.simulator import py_simulate_model
from bioscrape.types import Model

C:\Users\Jeremiah\anaconda3\lib\site-packages\biocrnpyler-0.2.1-py3.7.egg\biocrnpyler\__init__.py:37: UserWarning: No module named 'fa2'
C:\Users\Jeremiah\anaconda3\lib\site-packages\biocrnpyler-0.2.1-py3.7.egg\biocrnpyler\__init__.py:38: UserWarning: plotting is disabled because you are missing some libraries


In [2]:
# Create CRN for bistable switch
Core1_off = Species ("Core1")
Core2_off = Species ("Core2")

Core1 = Genelet(Core1_off, transcript = "rR1", activator = "dA2", inhibitor = "rR2" )
Core2 = Genelet(Core2_off, transcript = "rR2", activator = "dA1", inhibitor = "rR1" )
M_BiSwitch = Mixture(name = "Switch_test", components = [Core1, Core2], parameter_file = "default_parameters.txt")
repr(M_BiSwitch)
CRN_BiSwitch = M_BiSwitch.compile_crn()

# Write SBML file
CRN_BiSwitch.write_sbml_file('BiSwitch_CRN.xml')

print(CRN_BiSwitch)

Species = Core1_OFF, complex_Core1_ON, rna_rR1, dna_dA2, rna_rR2, protein_RNAP, protein_RNAseH, complex_Core1_AI, complex_Core1_ON_protein_RNAP, complex_Core1_OFF_protein_RNAP, complex_Core1_AI_protein_RNAseH, Core2_OFF, complex_Core2_ON, dna_dA1, complex_Core2_AI, complex_Core2_ON_protein_RNAP, complex_Core2_OFF_protein_RNAP, complex_Core2_AI_protein_RNAseH
Reactions = [
	Core1_OFF + dna_dA2 --> complex_Core1_ON        massaction: k_f(Core1_OFF,dna_dA2)=0.00394*Core1_OFF*dna_dA2
	complex_Core1_ON + rna_rR2 --> Core1_OFF + complex_Core1_AI        massaction: k_f(complex_Core1_ON,rna_rR2)=0.0696*complex_Core1_ON*rna_rR2
	dna_dA2 + rna_rR2 --> complex_Core1_AI        massaction: k_f(dna_dA2,rna_rR2)=0.0696*dna_dA2*rna_rR2
	complex_Core1_ON + protein_RNAP <--> complex_Core1_ON_protein_RNAP        massaction: k_f(complex_Core1_ON,protein_RNAP)=38.857142857142854*complex_Core1_ON*protein_RNAP k_r(complex_Core1_ON_protein_RNAP)=10.0*complex_Core1_ON_protein_RNAP
	complex_Core1_ON_protein_RNA

In [6]:
# Bioscrape simulation with one core ON and other core OFF

io = {"complex_Core1_ON": 0.1, "dna_dA2": 1, "rna_rR2": 0, "Core2_OFF": 0.1, "dna_dA1": 1, "rna_rR1": 0.5, "protein_RNAseH": 1,
      "protein_RNAP":15}
timepoints = np.linspace(0, 5000, 1000)
R = CRN_BiSwitch.simulate_with_bioscrape(timepoints, initial_condition_dict = io)

# For label convenience
x = 'Time (seconds)'
y = 'Concentration (uM)'

# Core 1 concentration/molecules
bokeh.io.output_notebook()
g = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
g.circle(timepoints, R["complex_Core2_ON"], legend_label = "ON Core 2", color = "red")
g.circle(timepoints, R["complex_Core1_ON"], legend_label = "ON Core 1", color = "blue")
g.legend.click_policy="hide"
g.legend.location = "center_right"

i = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
i.circle(timepoints, R["rna_rR1"], legend_label = "rR1", color = "red")
i.circle(timepoints, R["dna_dA1"], legend_label = "dA1", color = "orange")
i.legend.click_policy="hide"
i.legend.location = "center_right"

j = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
j.circle(timepoints, R["rna_rR2"], legend_label = "rR2", color = "blue")

bokeh.io.show(row(g, i, j))
warnings.filterwarnings("ignore")


Loading BokehJS ...

In [7]:
from bioscrape.sbmlutil import import_sbml
P = import_sbml ('Biswitch_CRN.xml')

#Initials

# M = Model(species = [Core1,Core2], parameters = "default_parameters.txt", reactions = P, initial_condition_dict = io)

P.set_species({"complex_Core1_ON": 0.1, "dna_dA2": 1, "rna_rR2": 0, "Core2_OFF": 0.1, "dna_dA1": 1, "rna_rR1": 0.5, "protein_RNAseH": 1,
      "protein_RNAP":15})
# Create parameters
P.create_parameter("rna_rR2i", 10)
P.create_parameter("T_rna_rR2i", 1800)

# Create rule 1
P.create_rule("assignment", {"equation":"rna_rR2 = _rna_rR2i*Heaviside(t-_T_rna_rR2i)"})

#Simulate Model through bioscrape
timepoints = np.linspace (0, 5000, 1000)
results = py_simulate_model (timepoints, Model = P)

# Plot Results


In [8]:
j = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
j.circle(timepoints, results["rna_rR2"], legend_label = "rR2", color = "blue")

k = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
k.circle(timepoints, results["complex_Core2_ON_protein_RNAP"], legend_label = "ON Core 2", color = "red")
k.circle(timepoints, results["complex_Core1_ON_protein_RNAP"], legend_label = "ON Core 1", color = "blue")
k.legend.click_policy="hide"
k.legend.location = "center_right"

m = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
m.circle(timepoints, results["rna_rR1"], legend_label = "rR1", color = "red")
m.circle(timepoints, results["dna_dA1"], legend_label = "dA1", color = "orange")
m.legend.click_policy="hide"
m.legend.location = "center_right"

bokeh.io.show(row(j, k, m))
warnings.filterwarnings("ignore")

export_png(row(k), filename="BiSwitch_Schaft_addrR2.png")

**Observations**
Changed to Schaffter and Schulman concentrations. Also, RNase H is not at proper concentration. Just set to 1.